# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [35]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [36]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab_to_int={}
    int_to_vocab={}
    for i,word in enumerate(set(text)):
        
        vocab_to_int[word]=i
        
    for w,i in vocab_to_int.items():
        int_to_vocab[i]=w
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [37]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    p_list = ['.',',','"',';','!','?','(',')','--','\n']
    p_words = ['||Period||','||comma||','||QuotationMark||','||Semicolon||',
              '||ExcalmationMark||','||QuestionMark||','||LeftParenthesis||',
              '||RightParenthesis||','||Dash||','||Return||']
    
    p_dict={}
    for i,p in enumerate(p_list):
        p_dict[p_list[i]]=p_words[i]
    return p_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [38]:
p_dict=token_lookup()
print(p_dict)

{'--': '||Dash||', '?': '||QuestionMark||', '!': '||ExcalmationMark||', '\n': '||Return||', '"': '||QuotationMark||', '.': '||Period||', '(': '||LeftParenthesis||', ')': '||RightParenthesis||', ';': '||Semicolon||', ',': '||comma||'}


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [42]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(dtype=tf.int32, shape=[None,None], name='input')
    targets = tf.placeholder(dtype=tf.int32, shape=[None,None], name='targets')
    learn_rate = tf.placeholder(dtype=tf.float64, name='learning_rate')
    return (inputs, targets, learn_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [61]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * 2)
    #cell = tf.contrib.rnn.MultiRNNCell([lstm_cell for _ in range(rnn_size)])
    initial_state=cell.zero_state(batch_size, tf.float32)
    
    return cell, tf.identity(initial_state, name="initial_state")


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [62]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [63]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(state,'final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [64]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data,vocab_size,embed_dim)
    #cell, intial_state = get_init_cell(input_data, rnn_size)
    out, final_state = build_rnn(cell,embed)
    logits = tf.contrib.layers.fully_connected(out, vocab_size,weights_initializer=tf.truncated_normal_initializer(stddev=0.1), activation_fn=None)
                          
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [65]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    nbatch = int(len(int_text)/(batch_size*seq_length))
    xdata = np.array(int_text[:nbatch*seq_length*batch_size])
    ydata = np.array(int_text[1:nbatch*seq_length*batch_size+1])
    ydata[-1] = xdata[0]
    batch_x = np.split(xdata.reshape(batch_size,-1),nbatch,1)
    batch_y = np.split(ydata.reshape(batch_size,-1),nbatch,1)
    batches = np.array(list(zip(batch_x,batch_y)))
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [72]:
# Number of Epochs
num_epochs = 320
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/53   train_loss = 8.835
Epoch   0 Batch    5/53   train_loss = 6.672
Epoch   0 Batch   10/53   train_loss = 6.775
Epoch   0 Batch   15/53   train_loss = 6.680
Epoch   0 Batch   20/53   train_loss = 6.586
Epoch   0 Batch   25/53   train_loss = 6.515
Epoch   0 Batch   30/53   train_loss = 6.463
Epoch   0 Batch   35/53   train_loss = 6.608
Epoch   0 Batch   40/53   train_loss = 6.321
Epoch   0 Batch   45/53   train_loss = 6.353
Epoch   0 Batch   50/53   train_loss = 6.344
Epoch   1 Batch    2/53   train_loss = 6.308
Epoch   1 Batch    7/53   train_loss = 6.255
Epoch   1 Batch   12/53   train_loss = 6.305
Epoch   1 Batch   17/53   train_loss = 6.244
Epoch   1 Batch   22/53   train_loss = 6.108
Epoch   1 Batch   27/53   train_loss = 6.212
Epoch   1 Batch   32/53   train_loss = 6.090
Epoch   1 Batch   37/53   train_loss = 6.376
Epoch   1 Batch   42/53   train_loss = 6.193
Epoch   1 Batch   47/53   train_loss = 6.223
Epoch   1 Batch   52/53   train_loss = 6.174
Epoch   2 

Epoch  17 Batch   14/53   train_loss = 4.293
Epoch  17 Batch   19/53   train_loss = 4.207
Epoch  17 Batch   24/53   train_loss = 4.205
Epoch  17 Batch   29/53   train_loss = 4.121
Epoch  17 Batch   34/53   train_loss = 4.307
Epoch  17 Batch   39/53   train_loss = 4.331
Epoch  17 Batch   44/53   train_loss = 4.286
Epoch  17 Batch   49/53   train_loss = 4.264
Epoch  18 Batch    1/53   train_loss = 4.208
Epoch  18 Batch    6/53   train_loss = 4.133
Epoch  18 Batch   11/53   train_loss = 4.292
Epoch  18 Batch   16/53   train_loss = 4.203
Epoch  18 Batch   21/53   train_loss = 4.114
Epoch  18 Batch   26/53   train_loss = 4.309
Epoch  18 Batch   31/53   train_loss = 4.238
Epoch  18 Batch   36/53   train_loss = 4.184
Epoch  18 Batch   41/53   train_loss = 4.225
Epoch  18 Batch   46/53   train_loss = 4.266
Epoch  18 Batch   51/53   train_loss = 4.162
Epoch  19 Batch    3/53   train_loss = 4.262
Epoch  19 Batch    8/53   train_loss = 4.060
Epoch  19 Batch   13/53   train_loss = 4.302
Epoch  19 

Epoch  34 Batch   28/53   train_loss = 3.150
Epoch  34 Batch   33/53   train_loss = 3.296
Epoch  34 Batch   38/53   train_loss = 3.296
Epoch  34 Batch   43/53   train_loss = 3.167
Epoch  34 Batch   48/53   train_loss = 3.231
Epoch  35 Batch    0/53   train_loss = 3.198
Epoch  35 Batch    5/53   train_loss = 3.223
Epoch  35 Batch   10/53   train_loss = 3.161
Epoch  35 Batch   15/53   train_loss = 3.191
Epoch  35 Batch   20/53   train_loss = 3.186
Epoch  35 Batch   25/53   train_loss = 3.308
Epoch  35 Batch   30/53   train_loss = 3.205
Epoch  35 Batch   35/53   train_loss = 3.194
Epoch  35 Batch   40/53   train_loss = 3.164
Epoch  35 Batch   45/53   train_loss = 3.087
Epoch  35 Batch   50/53   train_loss = 3.208
Epoch  36 Batch    2/53   train_loss = 3.187
Epoch  36 Batch    7/53   train_loss = 3.160
Epoch  36 Batch   12/53   train_loss = 3.196
Epoch  36 Batch   17/53   train_loss = 3.143
Epoch  36 Batch   22/53   train_loss = 3.099
Epoch  36 Batch   27/53   train_loss = 2.943
Epoch  36 

Epoch  51 Batch   42/53   train_loss = 2.170
Epoch  51 Batch   47/53   train_loss = 2.331
Epoch  51 Batch   52/53   train_loss = 2.396
Epoch  52 Batch    4/53   train_loss = 2.185
Epoch  52 Batch    9/53   train_loss = 2.305
Epoch  52 Batch   14/53   train_loss = 2.205
Epoch  52 Batch   19/53   train_loss = 2.148
Epoch  52 Batch   24/53   train_loss = 2.154
Epoch  52 Batch   29/53   train_loss = 2.320
Epoch  52 Batch   34/53   train_loss = 2.243
Epoch  52 Batch   39/53   train_loss = 2.230
Epoch  52 Batch   44/53   train_loss = 2.295
Epoch  52 Batch   49/53   train_loss = 2.306
Epoch  53 Batch    1/53   train_loss = 2.288
Epoch  53 Batch    6/53   train_loss = 2.272
Epoch  53 Batch   11/53   train_loss = 2.204
Epoch  53 Batch   16/53   train_loss = 2.200
Epoch  53 Batch   21/53   train_loss = 2.159
Epoch  53 Batch   26/53   train_loss = 2.173
Epoch  53 Batch   31/53   train_loss = 2.182
Epoch  53 Batch   36/53   train_loss = 2.267
Epoch  53 Batch   41/53   train_loss = 2.187
Epoch  53 

Epoch  69 Batch    3/53   train_loss = 1.583
Epoch  69 Batch    8/53   train_loss = 1.472
Epoch  69 Batch   13/53   train_loss = 1.465
Epoch  69 Batch   18/53   train_loss = 1.718
Epoch  69 Batch   23/53   train_loss = 1.709
Epoch  69 Batch   28/53   train_loss = 1.598
Epoch  69 Batch   33/53   train_loss = 1.647
Epoch  69 Batch   38/53   train_loss = 1.679
Epoch  69 Batch   43/53   train_loss = 1.523
Epoch  69 Batch   48/53   train_loss = 1.642
Epoch  70 Batch    0/53   train_loss = 1.551
Epoch  70 Batch    5/53   train_loss = 1.562
Epoch  70 Batch   10/53   train_loss = 1.542
Epoch  70 Batch   15/53   train_loss = 1.586
Epoch  70 Batch   20/53   train_loss = 1.625
Epoch  70 Batch   25/53   train_loss = 1.671
Epoch  70 Batch   30/53   train_loss = 1.584
Epoch  70 Batch   35/53   train_loss = 1.579
Epoch  70 Batch   40/53   train_loss = 1.661
Epoch  70 Batch   45/53   train_loss = 1.594
Epoch  70 Batch   50/53   train_loss = 1.634
Epoch  71 Batch    2/53   train_loss = 1.563
Epoch  71 

...

Epoch 241 Batch   37/53   train_loss = 0.923
Epoch 241 Batch   42/53   train_loss = 0.908
Epoch 241 Batch   47/53   train_loss = 0.890
Epoch 241 Batch   52/53   train_loss = 0.902
Epoch 242 Batch    4/53   train_loss = 0.928
Epoch 242 Batch    9/53   train_loss = 1.159
Epoch 242 Batch   14/53   train_loss = 0.919
Epoch 242 Batch   19/53   train_loss = 0.933
Epoch 242 Batch   24/53   train_loss = 1.104
Epoch 242 Batch   29/53   train_loss = 1.053
Epoch 242 Batch   34/53   train_loss = 1.100
Epoch 242 Batch   39/53   train_loss = 1.097
Epoch 242 Batch   44/53   train_loss = 1.118
Epoch 242 Batch   49/53   train_loss = 1.178
Epoch 243 Batch    1/53   train_loss = 1.054
Epoch 243 Batch    6/53   train_loss = 1.064
Epoch 243 Batch   11/53   train_loss = 1.204
Epoch 243 Batch   16/53   train_loss = 1.049
Epoch 243 Batch   21/53   train_loss = 1.020
Epoch 243 Batch   26/53   train_loss = 1.123
Epoch 243 Batch   31/53   train_loss = 1.058
Epoch 243 Batch   36/53   train_loss = 1.044
Epoch 243 

Epoch 258 Batch   51/53   train_loss = 0.296
Epoch 259 Batch    3/53   train_loss = 0.288
Epoch 259 Batch    8/53   train_loss = 0.276
Epoch 259 Batch   13/53   train_loss = 0.232
Epoch 259 Batch   18/53   train_loss = 0.282
Epoch 259 Batch   23/53   train_loss = 0.314
Epoch 259 Batch   28/53   train_loss = 0.298
Epoch 259 Batch   33/53   train_loss = 0.275
Epoch 259 Batch   38/53   train_loss = 0.291
Epoch 259 Batch   43/53   train_loss = 0.274
Epoch 259 Batch   48/53   train_loss = 0.283
Epoch 260 Batch    0/53   train_loss = 0.315
Epoch 260 Batch    5/53   train_loss = 0.271
Epoch 260 Batch   10/53   train_loss = 0.245
Epoch 260 Batch   15/53   train_loss = 0.246
Epoch 260 Batch   20/53   train_loss = 0.290
Epoch 260 Batch   25/53   train_loss = 0.278
Epoch 260 Batch   30/53   train_loss = 0.278
Epoch 260 Batch   35/53   train_loss = 0.243
Epoch 260 Batch   40/53   train_loss = 0.291
Epoch 260 Batch   45/53   train_loss = 0.257
Epoch 260 Batch   50/53   train_loss = 0.257
Epoch 261 

Epoch 276 Batch   12/53   train_loss = 0.246
Epoch 276 Batch   17/53   train_loss = 0.274
Epoch 276 Batch   22/53   train_loss = 0.310
Epoch 276 Batch   27/53   train_loss = 0.301
Epoch 276 Batch   32/53   train_loss = 0.272
Epoch 276 Batch   37/53   train_loss = 0.303
Epoch 276 Batch   42/53   train_loss = 0.275
Epoch 276 Batch   47/53   train_loss = 0.354
Epoch 276 Batch   52/53   train_loss = 0.314
Epoch 277 Batch    4/53   train_loss = 0.269
Epoch 277 Batch    9/53   train_loss = 0.309
Epoch 277 Batch   14/53   train_loss = 0.316
Epoch 277 Batch   19/53   train_loss = 0.327
Epoch 277 Batch   24/53   train_loss = 0.325
Epoch 277 Batch   29/53   train_loss = 0.367
Epoch 277 Batch   34/53   train_loss = 0.349
Epoch 277 Batch   39/53   train_loss = 0.345
Epoch 277 Batch   44/53   train_loss = 0.367
Epoch 277 Batch   49/53   train_loss = 0.336
Epoch 278 Batch    1/53   train_loss = 0.366
Epoch 278 Batch    6/53   train_loss = 0.355
Epoch 278 Batch   11/53   train_loss = 0.363
Epoch 278 

Epoch 293 Batch   26/53   train_loss = 0.452
Epoch 293 Batch   31/53   train_loss = 0.430
Epoch 293 Batch   36/53   train_loss = 0.425
Epoch 293 Batch   41/53   train_loss = 0.416
Epoch 293 Batch   46/53   train_loss = 0.400
Epoch 293 Batch   51/53   train_loss = 0.420
Epoch 294 Batch    3/53   train_loss = 0.416
Epoch 294 Batch    8/53   train_loss = 0.367
Epoch 294 Batch   13/53   train_loss = 0.344
Epoch 294 Batch   18/53   train_loss = 0.376
Epoch 294 Batch   23/53   train_loss = 0.460
Epoch 294 Batch   28/53   train_loss = 0.388
Epoch 294 Batch   33/53   train_loss = 0.409
Epoch 294 Batch   38/53   train_loss = 0.375
Epoch 294 Batch   43/53   train_loss = 0.387
Epoch 294 Batch   48/53   train_loss = 0.390
Epoch 295 Batch    0/53   train_loss = 0.384
Epoch 295 Batch    5/53   train_loss = 0.357
Epoch 295 Batch   10/53   train_loss = 0.340
Epoch 295 Batch   15/53   train_loss = 0.355
Epoch 295 Batch   20/53   train_loss = 0.377
Epoch 295 Batch   25/53   train_loss = 0.384
Epoch 295 

Epoch 310 Batch   40/53   train_loss = 0.231
Epoch 310 Batch   45/53   train_loss = 0.201
Epoch 310 Batch   50/53   train_loss = 0.188
Epoch 311 Batch    2/53   train_loss = 0.225
Epoch 311 Batch    7/53   train_loss = 0.244
Epoch 311 Batch   12/53   train_loss = 0.191
Epoch 311 Batch   17/53   train_loss = 0.197
Epoch 311 Batch   22/53   train_loss = 0.218
Epoch 311 Batch   27/53   train_loss = 0.190
Epoch 311 Batch   32/53   train_loss = 0.204
Epoch 311 Batch   37/53   train_loss = 0.206
Epoch 311 Batch   42/53   train_loss = 0.221
Epoch 311 Batch   47/53   train_loss = 0.222
Epoch 311 Batch   52/53   train_loss = 0.191
Epoch 312 Batch    4/53   train_loss = 0.197
Epoch 312 Batch    9/53   train_loss = 0.212
Epoch 312 Batch   14/53   train_loss = 0.198
Epoch 312 Batch   19/53   train_loss = 0.223
Epoch 312 Batch   24/53   train_loss = 0.204
Epoch 312 Batch   29/53   train_loss = 0.253
Epoch 312 Batch   34/53   train_loss = 0.223
Epoch 312 Batch   39/53   train_loss = 0.216
Epoch 312 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [75]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [76]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [77]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    
    inp = loaded_graph.get_tensor_by_name("input:0")
    ist = loaded_graph.get_tensor_by_name("initial_state:0")
    fst = loaded_graph.get_tensor_by_name("final_state:0")
    probis = loaded_graph.get_tensor_by_name("probs:0")
    return inp, ist, fst, probis



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [78]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    i = np.random.choice(list(int_to_vocab.keys()),p=probabilities)
    word = int_to_vocab[i]
    
    return str(word)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [79]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: legally, i can't say.
barney_gumble: well, you wanna wipe it, moe.
homer_simpson: you know, there's only one of the biggest time.
jacques: back, you've been replaced before you wanna walk in!
lenny_leonard: how did you don't want to keep your money uncle too far in.
lenny_leonard: huh? listen him.
homer_simpson:(chuckles)...(checks head) now it's going to happen to mean his wife to a cutie flaming homer" wasn't me so quickly!
homer_simpson:(counting carl, desperate) now i've been looking for this good for me?
moe_szyslak:(laughs) that's me.


moe_szyslak:(he glass) i hear, what are you still anything for milhouse.
homer_simpson: we don't have a great job, is that can you guys were just leaving, we all been at us.(perverted barflies)
homer_simpson:(sighs) then what do you get() crowd) and business) what are you, of is your father! uh..


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.